In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Importing training data

train_set = pd.read_csv("./data/Google_Stock_Price_Train.csv")
train_set = train_set.iloc[:, 1:2].values

In [ ]:
#Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
train_set = sc.fit_transform(train_set)

In [ ]:
#Get input and output
X_train = train_set[0:1257] #All the stock prices except the last one
y_train = train_set[1:1258]

#Reshaping needed to convert it to 3-D shape to be used by Keras modules
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train.shape

In [ ]:
#Import keras packages

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras import backend as K
K.set_image_dim_ordering('tf')

In [ ]:
#Initializing RNN
regressor = Sequential()
regressor.add(LSTM(4, activation = 'sigmoid', input_shape = (None, 1)))
regressor.add(Dense(1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

In [ ]:
regressor.fit(X_train, y_train, batch_size=32, nb_epoch = 200)

In [ ]:
test_set = pd.read_csv("./data/Google_Stock_Price_Test.csv")
real_stock_price = test_set.iloc[:, 1:2].values

In [ ]:
#Predict google stock price of Jan, 2017
inputs = real_stock_price

inputs = sc.transform(inputs)

inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))
inputs.shape

In [ ]:
predicted_stock_price = regressor.predict(inputs)
print(predicted_stock_price)

In [ ]:
#Reverse scale to get an actual value
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'red is real google stock price')
plt.plot(predicted_stock_price, color='blue', label = 'blue corresponds to predicted stock price')
plt.title("Google stock price prediction")
plt.xlabel("Time")
plt.ylabel("Google's stock price")
plt.legend()
plt.show()

In [ ]:
#Getting the real stock price of 2012 - 2016
real_stock_price_train = pd.read_csv("./data/Google_Stock_Price_Train.csv")
real_stock_price_train = real_stock_price_train.iloc[:, 1:2].values
#Getting the predicted stock prices of 2012 - 2016
predicted_stock_price_train = regressor.predict(X_train)
predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)

In [ ]:
plt.plot(real_stock_price_train, color = 'red', label = 'red is real google stock price')
plt.plot(predicted_stock_price_train, color='blue', label = 'blue corresponds to predicted stock price')
plt.title("Google stock price prediction")
plt.xlabel("Time")
plt.ylabel("Google's stock price")
plt.legend()
plt.show()

In [ ]:
#evaluating the RNN
import math
from sklearn.metrics import mean_squared_error 
rmse = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))

In [ ]:
print rmse

In [ ]:
def build_regressor(optimizer):
    regressor = Sequential()
    regressor.add(LSTM(4, activation = 'sigmoid', input_shape = (None, 1)))
    regressor.add(Dense(1))
    regressor.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics=['accuracy'])
    return regressor

In [ ]:
#Adding K-Fold validation to improve accuracy
from keras.wrappers.scikit_learn import KerasRegressor
regressor = KerasRegressor(build_fn = build_regressor)

In [ ]:
#Homework - create 4 RNN model with different timesteps, use parameter tuning , train and visualize result
from sklearn.model_selection import GridSearchCV
parameters = {'batch_size': [32, 64], 
              'nb_epoch': [50, 100],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = regressor, 
                           param_grid = parameters,
                           scoring = 'neg_mean_absolute_error',
                           cv = 10)

In [ ]:
#Fit the model using Gridsearch
grid_search.fit(X_train, y_train)

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
#Importing training data

train_set = pd.read_csv("./data/Google_Stock_Price_Train.csv")
train_set = train_set.iloc[:, 1:2].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
train_set_scaled = sc.fit_transform(train_set)
#Create data structure with 60 time steps and t+1 output
X_train = []
y_train = []
for i in range(60, 158):
    X_train.append(train_set_scaled[i-60:i, 0])
    y_train.append(train_set_scaled[i, 0])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
#Build Keras model and fit it
RNNRegressor = Sequential()
RNNRegressor.add(LSTM(units = 3, return_sequences = True, input_shape = (None , 1)))
RNNRegressor.add(LSTM(units = 3, return_sequences = True))
RNNRegressor.add(LSTM(units = 3, return_sequences = True))
RNNRegressor.add(LSTM(units = 3))
RNNRegressor.add(Dense(units = 1))
RNNRegressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
RNNRegressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
#Making the prediction and visualize the result

#Getting the real stock data
test_set = pd.read_csv("./data/Google_Stock_Price_Test.csv")
test_set = test_set.iloc([:, 1:2]).values
real_stock_price = np.concatenate((train_set[0:1258], test_set), axis = 0)

#getting predicted stock price of 2017
scaled_real_stock_price = sc.fit_transform(real_stock_price)
inputs = []
for i in range(1258, 1278):
    inputs.append(scaled_real_stock_price[i-60:i, 0])
inputs = np.array(inputs)
np.reshape(inputs, (inputs.shape[0], inputs_shape[1], 1))
predicted_stcok_price = RNNRegressor.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
#Plot the result
plt.plot(real_stock_price[1258:], color = 'red', label='Real Google Stock Price')
plt.plot(predicted_stock_price, color='blue', label="Predicted Google Stock Price")
plt.title('Google Stock Price Prediction')
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.legend()
plt.show()